In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import scipy.sparse as sp
import scipy.sparse import linalg

In [ ]:
import torch.nn.functional as F

class GATLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GATLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)
        self.attn_fc = nn.Linear(2 * out_features, 1, bias=False)

    def forward(self, input, adj):
        h = self.fc(input)
        N = h.size()[0]

        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.attn_fc(a_input).squeeze(2)

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        h_prime = torch.matmul(attention, h)

        return h_prime

class DCRNNGAT(nn.Module):
    def __init__(self, in_features, out_features, K):
        super(DCRNNGAT, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.K = K
        self.gat_layers = nn.ModuleList([GATLayer(in_features, out_features) for _ in range(K)])
        self.theta = nn.ParameterList([nn.Parameter(torch.FloatTensor(out_features, out_features)) for _ in range(K)])

    def forward(self, input, adj):
        adj = F.normalize(adj, p=1, dim=1)
        h = input
        for l in range(self.K):
            h_prime = self.gat_layers[l](h, adj)
            h = F.relu(torch.matmul(adj, self.theta[l] * h_prime))
        return h

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Define DCRNN parameters
num_nodes = 207  # number of nodes in the graph
max_diffusion_step = 1  # maximum diffusion step
num_rnn_layers = 2  # number of rnn layers
rnn_units = 64  # number of rnn units
num_epochs = 50  # number of training epochs
learning_rate = 0.001  # learning rate

# Define DCRNN cell
class DCRNNCell(layers.Layer):
    def __init__(self, num_nodes, max_diffusion_step, num_rnn_layers, rnn_units):
        super(DCRNNCell, self).__init__()
        self.num_nodes = num_nodes
        self.max_diffusion_step = max_diffusion_step
        self.rnn_layers = [layers.GRUCell(rnn_units) for _ in range(num_rnn_layers)]
        self.rnn = layers.RNN(self.rnn_layers, return_state=True)

    def call(self, inputs, states):
        # inputs: (batch_size, num_nodes * 2)
        # states: (num_rnn_layers, batch_size, rnn_units)
        seq_len = inputs.shape[1] // self.num_nodes // 2
        inputs = tf.reshape(inputs, (-1, seq_len, self.num_nodes, 2))
        outputs = []
        for diffusion_step in range(self.max_diffusion_step + 1):
            diffusion_inputs = inputs[..., diffusion_step]
            output, *states = self.rnn(diffusion_inputs, initial_state=states)
            outputs.append(output)
        outputs = tf.stack(outputs, axis=-1)
        return outputs, states

# Define DCRNN model
class DCRNN(tf.keras.Model):
    def __init__(self, num_nodes, max_diffusion_step, num_rnn_layers, rnn_units):
        super(DCRNN, self).__init__()
        self.dcrnn_cell = DCRNNCell(num_nodes, max_diffusion_step, num_rnn_layers, rnn_units)

    def call(self, inputs, states=None):
        return self.dcrnn_cell(inputs, states)

# Create DCRNN model
model = DCRNN(num_nodes, max_diffusion_step, num_rnn_layers, rnn_units)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse')

# Assume we have some data
x_train = np.random.rand(100, num_nodes * 2)
y_train = np.random.rand(100, num_nodes)

# Train model
model.fit(x_train, y_train, epochs=num_epochs)

Epoch 1/50


ValueError: in user code:

    C:\Users\vfgtr554\anaconda3\envs\test\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    C:\Users\vfgtr554\anaconda3\envs\test\lib\site-packages\tensorflow\python\keras\engine\training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\vfgtr554\anaconda3\envs\test\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\vfgtr554\anaconda3\envs\test\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\vfgtr554\anaconda3\envs\test\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\vfgtr554\anaconda3\envs\test\l

In [5]:
x_train

array([[0.16258483, 0.89049758, 0.49223242, ..., 0.49974541, 0.63361903,
        0.48984815],
       [0.13821947, 0.79235258, 0.42321294, ..., 0.54077391, 0.04677771,
        0.15554375],
       [0.12880391, 0.51516049, 0.983071  , ..., 0.20423396, 0.12769679,
        0.64615623],
       ...,
       [0.77267996, 0.56066875, 0.05483952, ..., 0.38290867, 0.49552517,
        0.43570257],
       [0.3374403 , 0.66657957, 0.96035318, ..., 0.12068194, 0.38866542,
        0.31795495],
       [0.83782522, 0.30069725, 0.88997837, ..., 0.42296577, 0.30218965,
        0.22763734]])

In [6]:
y_train

array([[0.20657844, 0.22275463, 0.68046933, ..., 0.00507025, 0.22786307,
        0.50851053],
       [0.78915376, 0.93901689, 0.16313523, ..., 0.52648473, 0.47959007,
        0.01088378],
       [0.26531201, 0.35840946, 0.78853034, ..., 0.77456382, 0.73763013,
        0.93017662],
       ...,
       [0.63552599, 0.34947106, 0.32205717, ..., 0.77110911, 0.01039287,
        0.9926409 ],
       [0.71472639, 0.16482182, 0.36440849, ..., 0.57960994, 0.6453524 ,
        0.36327981],
       [0.37715719, 0.71182306, 0.25937843, ..., 0.8508522 , 0.04120738,
        0.87402957]])

In [24]:
x_train.mean(), x_train.std(), y_train.mean(), y_train.std()

(0.4996038929869958,
 0.2881189346529228,
 0.5006716050315875,
 0.28919243755561674)

In [7]:
x_train.shape, y_train.shape

((100, 414), (100, 207))

In [12]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [17]:
x_train.shape[1] // 207 // 2

1

In [ ]:
inputs.shape[1] // self.num_nodes // 2

